In [1]:
import numpy as np
import pandas as pd
import os
import face_recognition
import joblib
import ipywidgets as widgets
from IPython.display import display 
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../Desktop/illinois/person.csv", sep = ";")
pd.set_option("display.max_columns", None)
df = df.dropna(subset=['weight', 'height'])
df = df.drop(columns=['date_of_birth', 'hair', 'sex', 'race', 'eyes', 'admission_date', 'projected_parole_date', 
              'last_paroled_date', 'projected_discharge_date', 'parole_date', 'electronic_detention_date', 
              'discharge_date', 'parent_institution', 'offender_status', 'location', 'sex_offender_registry_required',
             'alias', 'Unnamed: 21'])
df = df.reset_index(drop=True)

df['heightininch'] = 0
for i in range(len(df)):
    df['heightininch'][i] = int(str(df['height'][i])[:1]) * 12 + int(float(str(df['height'][i])[1:]))

df['bmi'] = df['weight'] * 703 / df['heightininch']**2


C:\Users\oswal\AppData\Local\Temp\ipykernel_42372\83835706.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['heightininch'][i] = int(str(df['height'][i])[:1]) * 12 + int(float(str(df['height'][i])[1:]))


In [3]:
dfow = df.loc[(df['bmi'] >= 25) & (df['bmi'] < 30)]
dfob = df.loc[(df['bmi'] >= 30)]
dfno = df.loc[(df['bmi'] >= 18.5) & (df['bmi'] < 25)]
dfuw = df.loc[(df['bmi'] < 18.5)]
dfuw = dfuw.reset_index(drop=True)
dfno = dfno.reset_index(drop=True)
dfow = dfow.reset_index(drop=True)
dfob = dfob.reset_index(drop=True)
dfob = dfob.drop(dfob.index[900:])
dfow = dfow.drop(dfow.index[900:])
dfno = dfno.drop(dfno.index[900:])
dfuw = dfuw.drop(dfuw.index[900:])

In [4]:
df_merged = pd.concat([dfow, dfno, dfob, dfuw], ignore_index=True, sort=False)
df_merged.head()

,id,name,weight,height,heightininch,bmi
0,A01457,"HAYES, LARRY",231.0,68.0,80,25.373906
1,A01950,"BROWN, RALPH",259.0,69.0,81,27.751410
2,A08141,"COWAN, EARL L.",300.0,70.0,84,29.889456
3,A10309,"BUTLER, RICHARD",202.0,60.0,72,27.393133
4,A10695,"MACLIN, GEORGE",248.0,69.0,81,26.572779


In [5]:
def face_into_encoding(path):
#     try:
    image = face_recognition.load_image_file(path)
    face_encoding = face_recognition.face_encodings(image)
    if not face_encoding:
        print("skipped" + path)
        return np.zeros(128).tolist()
    return face_encoding[0].tolist()

In [6]:
face_encoded_list = []
for imag in df_merged.id:
    face_encoded = face_into_encoding('../Desktop/illinois/front/front/'+imag+'.jpg')
    face_encoded_list.append(face_encoded)

skipped../Desktop/illinois/front/front/A80582.jpg
skipped../Desktop/illinois/front/front/B08176.jpg
skipped../Desktop/illinois/front/front/B09641.jpg
skipped../Desktop/illinois/front/front/B12871.jpg
skipped../Desktop/illinois/front/front/B20741.jpg
skipped../Desktop/illinois/front/front/B36789.jpg
skipped../Desktop/illinois/front/front/B44617.jpg
skipped../Desktop/illinois/front/front/B46822.jpg
skipped../Desktop/illinois/front/front/B69219.jpg
skipped../Desktop/illinois/front/front/B71273.jpg
skipped../Desktop/illinois/front/front/C01435.jpg
skipped../Desktop/illinois/front/front/A63742.jpg
skipped../Desktop/illinois/front/front/A70170.jpg
skipped../Desktop/illinois/front/front/A70618.jpg
skipped../Desktop/illinois/front/front/A74143.jpg
skipped../Desktop/illinois/front/front/A90240.jpg
skipped../Desktop/illinois/front/front/A91250.jpg
skipped../Desktop/illinois/front/front/A92280.jpg
skipped../Desktop/illinois/front/front/B04516.jpg
skipped../Desktop/illinois/front/front/B05409.jpg


In [7]:
X = np.array(face_encoded_list)

In [8]:
y = df_merged.bmi.values
plotX = pd.DataFrame(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [11]:
rf_model = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
rf_model = rf_model.fit(X_train, np.log(y_train))
pred = rf_model.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, np.exp(pred))))

Mean Absolute Error: 4.499776227721669


In [12]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
bootstrap = [True, False]
criterion = ['absolute_error']
max_features = ['sqrt', 'auto', 'log2']
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [1, 2, 4, 5]
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

rf_grid = {'n_estimators': n_estimators,
            'bootstrap': bootstrap,
            'criterion': criterion,
            'max_features': max_features,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'max_depth': max_depth}

In [13]:
rf_model_tuned = RandomizedSearchCV(estimator=RandomForestRegressor(), param_distributions=rf_grid, n_iter=100, cv=3, random_state=42, n_jobs=-1)
rf_model_tuned.fit(X_train, np.log(y_train))
predictions = rf_model_tuned.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, np.exp(predictions))))

Mean Absolute Error: 3.8962102613738594


In [14]:
rf_model = '../Desktop/rf_bmi_model_tuned'
joblib.dump(rf_model_tuned, rf_model)
model = joblib.load(rf_model)

In [15]:
def predict_bmi(X_images, model):
    X = np.expand_dims(np.array(face_into_encoding(X_images)), axis=0)
    log_value = model.predict(X)
    bmi = np.exp(log_value)
    return bmi

In [30]:
images = os.listdir('../Desktop/ttrial')
for img in images:
    img_location = '../Desktop/ttrial/' + img
    img = open(img_location, 'rb')
    test_img = img.read()
    preds = predict_bmi(img_location, model)
    display(widgets.Image(value=test_img, format='jpg', width=200, height=200))
    print("Your BMI is: " + str(preds) + "kg/m2")

Image(value=b'\xff\xd8\xff\xe1\x00\x18Exif\x00\x00II*\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\…

Your BMI is: [30.86486546]kg/m2


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00;CREATOR: gd-jpe…

Your BMI is: [22.394855]kg/m2


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x06\x04\x0…

...

Your BMI is: [19.3226774]kg/m2


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe1\x00tExif\x00\x00II*\x00\x…

Your BMI is: [24.3679716]kg/m2


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Your BMI is: [25.58293827]kg/m2
